# McKinsey hackaton 


In [219]:
import numpy as np                              
import pandas as pd                              
import matplotlib.pyplot as plt                  
import seaborn as sns                            

%matplotlib inline

from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor 
from sklearn.model_selection import cross_val_score

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [220]:
train = pd.read_csv('data/train_data.csv', index_col=['year'], parse_dates=['year'])
test = pd.read_csv('data/test_data.csv', index_col=['year'], parse_dates=['year'])

# Посмотрим на данные

In [221]:
train.head()

,robot_gear_compression_diff_1,sensor_robot_controller_index_2,robot_gear_compression_diff_3,robot_gear_compression_diff_4,sensor_robot_arm_right_1,robot_gear_compression_diff_6,robot_gear_compression_diff_7,robot_gear_compression_diff_8,robot_gear_compression_diff_9,robot_gear_compression_diff_10,...,robot_probe_temperature_5,robot_probe_temperature_6,robot_probe_temperature_7,robot_probe_temperature_8,robot_probe_temperature_9,sensor_robot_eye_laser_range_1,sensor_robot_arm_left_4,sensor_robot_arm_left_2,gamma_ray,target
year,,,,,,,,,,,,,,,,,,,,,
2019,10.679910,15.091173,14.274428,-11.045572,-4.783722,8.581814,13.683193,5.829388,-46.055115,23.933078,...,-19.814141,-25.839817,0.970255,3.238174,1.817238,1.474627,-28.059812,15.631746,low,19.993461
2020,10.611032,14.906948,13.773004,-11.169123,-6.077731,8.336961,13.633622,5.345224,-50.708744,28.812526,...,-0.198837,-7.075529,0.728076,3.004283,1.586136,1.281429,-27.609630,15.848169,low,19.904883
2021,10.186666,14.807848,13.861197,-12.261832,-5.385449,8.208332,14.534426,5.352057,-53.471611,25.629468,...,0.142533,-4.374839,0.397508,3.050232,2.093176,1.791260,-25.408680,16.064592,low,20.422236
2022,10.439354,14.780703,13.735109,-12.940010,-3.136724,8.093524,14.911227,7.367078,-42.954456,28.717202,...,-0.881582,-3.638186,2.683104,3.133879,1.799235,1.588151,-23.169179,16.281016,low,19.872086
2023,10.537853,14.761712,14.112636,-11.648818,-1.120589,8.359302,14.660116,7.028476,-49.266612,29.162689,...,2.963216,-2.100919,0.409917,3.240069,1.315912,0.893747,-22.344542,16.497439,low,19.118784


In [222]:
train.describe()

,robot_gear_compression_diff_1,sensor_robot_controller_index_2,robot_gear_compression_diff_3,robot_gear_compression_diff_4,sensor_robot_arm_right_1,robot_gear_compression_diff_6,robot_gear_compression_diff_7,robot_gear_compression_diff_8,robot_gear_compression_diff_9,robot_gear_compression_diff_10,...,sensor_robot_eye_laser_sensor_4,robot_probe_temperature_5,robot_probe_temperature_6,robot_probe_temperature_7,robot_probe_temperature_8,robot_probe_temperature_9,sensor_robot_eye_laser_range_1,sensor_robot_arm_left_4,sensor_robot_arm_left_2,target
count,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,...,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000,3263.000000
mean,4.743382,4.711188,4.150240,0.735772,0.356302,3.980784,4.795702,3.489475,-11.762066,6.551856,...,-0.314341,-1.619153,-1.227771,-0.368780,-0.966306,-0.351048,-0.102309,-3.162996,-5.764416,3.974753
std,11.971591,12.600201,15.414344,32.491776,21.249066,11.971590,21.787280,23.439238,39.628929,22.635889,...,13.785414,15.729530,15.046405,11.752485,52.007397,25.875938,19.137775,13.069460,48.923364,21.058748
min,-41.222606,-50.708097,-85.115629,-237.802677,-62.005399,-20.918665,-37.047343,-30.209884,-129.428980,-60.284484,...,-260.782205,-204.598022,-267.461044,-171.454268,-700.034237,-350.057140,-259.983389,-45.065021,-260.177848,-222.068373
25%,-4.347614,-5.053198,-5.476476,-11.600617,-14.192575,-3.085307,-7.531737,-7.319708,-37.700655,-11.815518,...,0.528241,-2.906805,-1.409485,-0.300321,3.006966,1.331924,1.089518,-12.711290,-21.939303,-6.439736
50%,10.982071,11.185722,12.667528,2.585477,6.194435,6.779509,10.790315,3.339351,-15.966160,17.058741,...,0.813811,4.115493,2.352888,0.757984,3.105589,1.690868,1.386072,-2.944265,-2.201517,10.087139
75%,14.536056,15.194974,15.871684,23.110990,17.123009,11.240535,16.412493,13.688108,16.612456,23.523683,...,0.956559,6.543507,4.231996,1.647482,3.193184,2.047907,1.675096,6.607371,18.228823,17.292415
max,16.485844,17.096144,20.793896,65.593323,126.019083,122.988116,242.904661,308.146334,112.222819,40.792290,...,1.813343,17.959283,12.010494,3.795320,6.940355,4.374593,51.588361,30.050211,98.218800,25.643372


In [223]:
train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3263 entries, 2019 to 5281
Data columns (total 145 columns):
robot_gear_compression_diff_1        3263 non-null float64
sensor_robot_controller_index_2      3263 non-null float64
robot_gear_compression_diff_3        3263 non-null float64
robot_gear_compression_diff_4        3263 non-null float64
sensor_robot_arm_right_1             3263 non-null float64
robot_gear_compression_diff_6        3263 non-null float64
robot_gear_compression_diff_7        3263 non-null float64
robot_gear_compression_diff_8        3263 non-null float64
robot_gear_compression_diff_9        3263 non-null float64
robot_gear_compression_diff_10       3263 non-null float64
robot_gear_circulation_1             3263 non-null float64
robot_gear_circulation_2             3263 non-null float64
sensor_robot_arm_left_3              3263 non-null float64
sensor_robot_controller_index_5      3263 non-null float64
sensor_robot_controller_index_3      3263 non-null float64
robo

In [224]:
test.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 5282 to 6281
Data columns (total 145 columns):
robot_gear_compression_diff_1        1000 non-null float64
sensor_robot_controller_index_2      1000 non-null float64
robot_gear_compression_diff_3        1000 non-null float64
robot_gear_compression_diff_4        1000 non-null float64
sensor_robot_arm_right_1             1000 non-null float64
robot_gear_compression_diff_6        1000 non-null float64
robot_gear_compression_diff_7        1000 non-null float64
robot_gear_compression_diff_8        1000 non-null float64
robot_gear_compression_diff_9        1000 non-null float64
robot_gear_compression_diff_10       1000 non-null float64
robot_gear_circulation_1             1000 non-null float64
robot_gear_circulation_2             1000 non-null float64
sensor_robot_arm_left_3              1000 non-null float64
sensor_robot_controller_index_5      1000 non-null float64
sensor_robot_controller_index_3      1000 non-null float64
robo

In [225]:
train['gamma_ray'].unique()

array(['low', 'moderate', 'high'], dtype=object)

# Preprocessing

In [226]:
def preprocessing(data, train=True):
    data = pd.get_dummies(data, columns=['gamma_ray'])
    
    # По идее, это относительные парамтеры very_low < low < medium etc
    # TO DO: пробовать перевести значения в 1,2,3,4,...
    if 'gamma_ray_very high' not in data.columns:
        data['gamma_ray_very high'] = 0
    if 'gamma_ray_low' not in data.columns:
        data['gamma_ray_low'] = 0
    if 'gamma_ray_moderate' not in data.columns:
        data['gamma_ray_moderate'] = 0
    
    #TO DO: раскомментировать строки ниже
    # Пока убрал работу с временными лагами, т.к. не понятно, что делать в момент предикта (нужно всегда будет обновлять двнные предиктом для наполнения лагов следующего шага)
#     #Посмотрим, влияют ли как-то предыдущие значения таргета на последующее (TO DO: попробовать без лага)
#     for i in range(1, 10):
#         data["lag_{}".format(i)] = data.target.shift(i).fillna(0) #первые значения лага заполняем 0 (TO DO: попробовать среднее значение и Nan)
    
    if train:
        target = data['target'] 
        data.drop(columns=['target'], inplace=True)
    else:
        target =[]
    return data, target

In [227]:
x_train, y_train = preprocessing(train)

In [228]:
x_train.head()

,robot_gear_compression_diff_1,sensor_robot_controller_index_2,robot_gear_compression_diff_3,robot_gear_compression_diff_4,sensor_robot_arm_right_1,robot_gear_compression_diff_6,robot_gear_compression_diff_7,robot_gear_compression_diff_8,robot_gear_compression_diff_9,robot_gear_compression_diff_10,...,robot_probe_temperature_7,robot_probe_temperature_8,robot_probe_temperature_9,sensor_robot_eye_laser_range_1,sensor_robot_arm_left_4,sensor_robot_arm_left_2,gamma_ray_high,gamma_ray_low,gamma_ray_moderate,gamma_ray_very high
year,,,,,,,,,,,,,,,,,,,,,
2019,10.679910,15.091173,14.274428,-11.045572,-4.783722,8.581814,13.683193,5.829388,-46.055115,23.933078,...,0.970255,3.238174,1.817238,1.474627,-28.059812,15.631746,0,1,0,0
2020,10.611032,14.906948,13.773004,-11.169123,-6.077731,8.336961,13.633622,5.345224,-50.708744,28.812526,...,0.728076,3.004283,1.586136,1.281429,-27.609630,15.848169,0,1,0,0
2021,10.186666,14.807848,13.861197,-12.261832,-5.385449,8.208332,14.534426,5.352057,-53.471611,25.629468,...,0.397508,3.050232,2.093176,1.791260,-25.408680,16.064592,0,1,0,0
2022,10.439354,14.780703,13.735109,-12.940010,-3.136724,8.093524,14.911227,7.367078,-42.954456,28.717202,...,2.683104,3.133879,1.799235,1.588151,-23.169179,16.281016,0,1,0,0
2023,10.537853,14.761712,14.112636,-11.648818,-1.120589,8.359302,14.660116,7.028476,-49.266612,29.162689,...,0.409917,3.240069,1.315912,0.893747,-22.344542,16.497439,0,1,0,0


# Обучение модели 

In [229]:
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # считаем индекс в датафрейме, после которого начинается тестовый отрезок
    test_index = int(len(X)*(1-test_size))
    
    # разбиваем весь датасет на тренировочную и тестовую выборку
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [230]:
X_tr, X_te, y_tr, y_te = timeseries_train_test_split(x_train, y_train, test_size=0.3)

In [231]:
X_tr.shape, X_te.shape

((2284, 147), (979, 147))

## Random Forest

In [232]:
rf = RandomForestRegressor(random_state=46, n_estimators=55)

In [233]:
rf.fit(X_tr, y_tr)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=55, n_jobs=None,
           oob_score=False, random_state=46, verbose=0, warm_start=False)

In [234]:
cross_val_score(
    rf,
    X_te,
    y_te,
    scoring='r2',
    cv=10
)

array([ 0.88936431,  0.8868314 ,  0.90808053,  0.83367282,  0.90488115,
        0.80917767,  0.76783782,  0.86581094, -0.12207047, -0.18783668])

## XGBoost

In [235]:
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [236]:
cross_val_score(
    xgb,
    X_te,
    y_te,
    scoring='r2',
    cv=10
)

array([ 0.83315053,  0.91341524,  0.92502848,  0.81375512,  0.91232641,
        0.7853384 ,  0.68479056,  0.89133386, -0.12085051,  0.89766269])

## Формируем результаты

In [237]:
X_test,_ = preprocessing(test)

In [238]:
X_test.head()

,robot_gear_compression_diff_1,sensor_robot_controller_index_2,robot_gear_compression_diff_3,robot_gear_compression_diff_4,sensor_robot_arm_right_1,robot_gear_compression_diff_6,robot_gear_compression_diff_7,robot_gear_compression_diff_8,robot_gear_compression_diff_9,robot_gear_compression_diff_10,...,robot_probe_temperature_7,robot_probe_temperature_8,robot_probe_temperature_9,sensor_robot_eye_laser_range_1,sensor_robot_arm_left_4,sensor_robot_arm_left_2,gamma_ray_high,gamma_ray_very high,gamma_ray_low,gamma_ray_moderate
year,,,,,,,,,,,,,,,,,,,,,
5282,-18.136584,-18.926809,-20.810190,-26.048782,-6.143792,-13.271357,-22.288341,-19.924976,-17.556931,-31.181639,...,2.556148,3.212842,1.454573,1.223251,15.791951,7.321099,1,0,0,0
5283,-20.960127,-22.085946,-26.493563,-46.317078,-16.710923,-15.651756,-26.447869,-22.773740,-31.982001,-37.597457,...,0.930479,3.236609,0.868422,0.855648,18.934608,7.710661,1,0,0,0
5284,-22.938643,-23.746999,-28.380429,-54.863228,-19.916516,-17.705687,-30.146180,-25.755859,-29.190648,-38.858731,...,1.667303,3.092737,1.268653,1.206389,19.788746,8.100223,1,0,0,0
5285,-22.790936,-23.812049,-28.283436,-56.067944,-19.997358,-18.777231,-32.326135,-27.382583,-18.938983,-38.015383,...,-0.854167,3.055166,2.466574,1.559834,20.566138,8.489784,1,0,0,0
5286,-23.167947,-24.002717,-29.013752,-57.311637,-17.371633,-18.488637,-31.337333,-25.640987,-13.643481,-37.172163,...,2.712196,3.124018,1.860862,1.660488,20.666446,8.879346,1,0,0,0


In [239]:
result = xgb.predict(X_test)

ValueError: feature_names mismatch: ['robot_gear_compression_diff_1', 'sensor_robot_controller_index_2', 'robot_gear_compression_diff_3', 'robot_gear_compression_diff_4', 'sensor_robot_arm_right_1', 'robot_gear_compression_diff_6', 'robot_gear_compression_diff_7', 'robot_gear_compression_diff_8', 'robot_gear_compression_diff_9', 'robot_gear_compression_diff_10', 'robot_gear_circulation_1', 'robot_gear_circulation_2', 'sensor_robot_arm_left_3', 'sensor_robot_controller_index_5', 'sensor_robot_controller_index_3', 'robot_gear_circulation_6', 'sensor_robot_arm_right_4', 'robot_gear_circulation_8', 'sensor_robot_arm_right_2', 'sensor_robot_jump_power_4', 'sensor_robot_jump_power_5', 'robot_gear_circulation_12', 'robot_gear_circulation_13', 'robot_gear_circulation_14', 'robot_gear_circulation_15', 'sensor_robot_jump_power_3', 'sensor_robot_arm_left_5', 'robot_gear_circulation_18', 'robot_gear_circulation_19', 'robot_gear_circulation_20', 'robot_gear_circulation_21', 'sensor_robot_arm_right_